In [9]:
from pynq import Overlay
from pynq import MMIO

Max_buffer_size = 345600
Clefia = Overlay("./bitstream/256/design_dec.bit")

phys_addr = Clefia.ip_dict["clefia_dec_0"]["phys_addr"]
addr_range = Clefia.ip_dict["clefia_dec_0"]["addr_range"]
mmio = MMIO(phys_addr, addr_range)

#Clefia.ip_dict #offsetの確認

In [10]:
def call_Clefia(pt, skey):
    mmio.write(16,bytes(pt))            # Memory_pt: 16
    mmio.write(32,bytes(skey))          # Memory_skey: 32
    mmio.write(0,1)                     # ap_start: 0
    while not (mmio.read(0) & 0x02):
        pass

    return 0



In [11]:
import time
# byte型はmmioに一度ですべて書き込める
# for 128 bit
#ct = bytes([0xde,0x2b,0xf2,0xfd,0x9b,0x74,0xaa,0xcd,0xf1,0x29,0x85,0x55,0x45,0x94,0x94,0xfd])
# for 192 bit
#ct = bytes([0xe2,0x48,0x2f,0x64,0x9f,0x02,0x8d,0xc4,0x80,0xdd,0xa1,0x84,0xfd,0xe1,0x81,0xad])
# for 256 bit
ct = bytes([0xa1,0x39,0x78,0x14,0x28,0x9d,0xe8,0x0c,0x10,0xda,0x46,0xd1,0xfa,0x48,0xb3,0x8a])

skey = bytes([0xff,0xee,0xdd,0xcc,0xbb,0xaa,0x99,0x88,
         0x77,0x66,0x55,0x44,0x33,0x22,0x11,0x00,
         0xf0,0xe0,0xd0,0xc0,0xb0,0xa0,0x90,0x80,
         0x70,0x60,0x50,0x40,0x30,0x20,0x10,0x00])

print(ct)  # x09, x0a, x0dはASCIIで置き換えられる（数値に影響はなし）

NUM_OF_LOOPS = 1<<17
i = 0

# -----計算開始-----
time_start = time.perf_counter_ns()
for i in range(0, NUM_OF_LOOPS, 1):
    call_Clefia(ct, skey)
    
time_end = time.perf_counter_ns()
# -----計算終了-----

time_took = time_end - time_start
time_ave = time_took / NUM_OF_LOOPS

print(f"Average:{time_ave}ns")



b'\xa19x\x14(\x9d\xe8\x0c\x10\xdaF\xd1\xfaH\xb3\x8a'
Average:157086.0067062378ns


In [8]:
string = "abcdefgh" #Max 16 Bytes & Multiple of 4
print(len(string.encode()))
string = bytes(string.encode())
key_bitlen = 128

call_Clefia(string, key_bitlen)

"""stringへの入力が4バイト単位しか受け付けない
   空白文字を入れるなどの処理をする？？？"""
"""ポートのサイズを拡大したらもっと処理できる？？
Pythonで16バイトずつ区切って何度か呼び出す？"""

8

-----処理時間計測開始-----
計算前： 4
計算中： 1
waiting...
計算中： 4
waiting...
計算後： 4

-----処理時間計測終了-----
計算時間： 0.0009067058563232422 s.
ciphertext: [b'1\x95\x16\x1d', b'\xfc\xdbK\x95', b'f,\xe7\r', b'\x04\xfd\xee)']
plaintext: [b'abcd', b'efgh', b'\x00\x00\x00\x00', b'\x00\x00\x00\x00']


'ポートのサイズを拡大したらもっと処理できる？？\nPythonで16バイトずつ区切って何度か呼び出す？'

In [ ]:
def call_Clefia(pt):
    temp = 0
    dec = []
    print("\n-----処理時間計測開始-----") #消す
    time_start = time.perf_counter_ns()
    mmio.write(16,bytes(pt))            # Memory_pt: 16
    print("計算前：", mmio.read(CTRL)) #消す
    mmio.write(0,1)                     # ap_start: 0
    while not mmio.read(0x00 & 0x04) == 0x04: 
        print("計算中：", mmio.read(CTRL)) #消す
        print("waiting...") # passに変える
        
    print("計算後：", mmio.read(CTRL)) #消す
    time_end = time.perf_counter_ns()
    print("\n-----処理時間計測終了-----") #消す
    print("計算時間：", time_end-time_start, "s.")
    for i in range(0,4): #一度に１６バイトの読み込みはできない
        dec.append(mmio.read(32+(i*4)).to_bytes(4,"little"))  #Memory_Clefia_enc: 32
          
    print("plaintext:",dec)
    return 0

